In [224]:
# data manipulation
import pandas as pd
import numpy as np

# data viz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


import chime

In [225]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [226]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [227]:
print('-'*20, 'train data', '-'*20)
train_df.info()

-------------------- train data --------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Correlation of target and categorical features

In [228]:
train_df[['Pclass','Survived']].groupby(['Pclass'], as_index=False).mean()

# passenger class has impact on survivability

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [229]:
train_df[['Embarked','Survived']].groupby(['Embarked'], as_index=False).mean()

# embark has impact on survivability

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [230]:
# creating 2 new features
# - combining SibSp and Parch to total number of family members
# - the passenger is alone on the ship
train_df['NumRelatives'] = train_df['SibSp'] + train_df['Parch']

train_df['Alone'] = np.where(train_df.NumRelatives == 0, 1, 0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NumRelatives,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1


In [231]:
train_df[['NumRelatives','Survived']].groupby(['NumRelatives'], as_index=False).mean('Survived')

# number of relatives has low impact on survivability

,NumRelatives,Survived
0,0,0.303538
1,1,0.552795
2,2,0.578431
3,3,0.724138
4,4,0.200000
5,5,0.136364
6,6,0.333333
7,7,0.000000
8,10,0.000000


In [232]:
train_df[['Alone','Survived']].groupby(['Alone'], as_index=False).mean('Survived')

# flag alone has good impact on survivability

,Alone,Survived
0,0,0.505650
1,1,0.303538


In [233]:
# drop NumRelatives and replicate Alone on test data
train_df = train_df.drop('NumRelatives', axis=1)

test_df['Alone'] = np.where(test_df.SibSp + test_df.Parch == 0, 1, 0)

In [234]:
# drop useless columns
train_df = train_df.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch'], axis=1)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Alone
0,0,3,male,22.0,7.2500,S,0
1,1,1,female,38.0,71.2833,C,0
2,1,3,female,26.0,7.9250,S,1
3,1,1,female,35.0,53.1000,S,0
4,0,3,male,35.0,8.0500,S,1


In [235]:
# encoding categorical features
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
# train_df['Embarked'] = train_df['Embarked'].map({'S': 1, 'Q': 2, 'C': 3}).astype(int)



In [236]:
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Alone
0,0,3,0,22.0,7.2500,S,0
1,1,1,1,38.0,71.2833,C,0
2,1,3,1,26.0,7.9250,S,1
3,1,1,1,35.0,53.1000,S,0
4,0,3,0,35.0,8.0500,S,1


In [237]:
ages = np.zeros((2,3))

def fill_age(df):

    for i in range(0, 2):
        for j in range(0, 3):

            temp_df = df[(df['Sex'] == i) & (df['Pclass'] == j+1)]['Age'].dropna()

            temp_age = temp_df.median()
            ages[i,j] = int(temp_age/0.5 + 0.5)*0.5

    for i in range(0, 2):
        for j in range(0, 3):
            df.loc[ (df.Age.isnull()) & (df.Sex == i) & (df.Pclass == j+1), 'Age' ] = ages[i,j]

    df['Age'] = df['Age'].astype(int)

    return df

In [238]:
train_df = fill_age(train_df)
test_df = fill_age(test_df)

train_df['AgeRange'] = pd.cut(train_df['Age'], 5)

train_df[['AgeRange','Survived']].groupby(['AgeRange'], as_index=False).mean()

,AgeRange,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.337374
2,"(32.0, 48.0]",0.412037
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [239]:
def encode_age(df):
    df.loc[ df['Age'] <= 16, 'Age'] = 0
    df.loc[ (df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
    df.loc[ (df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
    df.loc[ (df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
    df.loc[ (df['Age'] > 64), 'Age'] = 4
    
    return df

train_df = encode_age(train_df).drop('AgeRange', axis = 1)
test_df = encode_age(test_df)

train_df.head()


,Survived,Pclass,Sex,Age,Fare,Embarked,Alone
0,0,3,0,1,7.2500,S,0
1,1,1,1,2,71.2833,C,0
2,1,3,1,1,7.9250,S,1
3,1,1,1,2,53.1000,S,0
4,0,3,0,2,8.0500,S,1


In [240]:
# filling null values on enbarked
train_df['Embarked'] = train_df['Embarked'].fillna('S')

# encoding 
train_df['Embarked'] = train_df['Embarked'].map({'S': 0, 'Q': 1, 'C': 2}).astype(int)
test_df['Embarked'] = test_df['Embarked'].map({'S': 0, 'Q': 1, 'C': 2}).astype(int)

In [241]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

train_df['FareRange'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareRange', 'Survived']].groupby(['FareRange'], as_index=False).mean()

,FareRange,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [242]:
def encode_fare(df):
    df.loc[ df['Fare'] <= 7.91, 'Fare'] = 0
    df.loc[ (df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
    df.loc[ (df['Fare'] > 14.454) & (df['Fare'] <= 31.0), 'Fare'] = 2
    df.loc[ (df['Fare'] > 31.0), 'Fare'] = 3
    df['Fare'] = df['Fare'].astype(int)
    
    return df

train_df = encode_fare(train_df).drop('FareRange', axis = 1)
test_df = encode_fare(test_df)

train_df.head()


,Survived,Pclass,Sex,Age,Fare,Embarked,Alone
0,0,3,0,1,0,0,0
1,1,1,1,2,3,2,0
2,1,3,1,1,1,0,1
3,1,1,1,2,3,0,0
4,0,3,0,2,1,0,1


In [243]:
test_df.head()

,Pclass,Sex,Age,Fare,Embarked,Alone
0,3,0,2,0,1,1
1,3,1,2,0,0,0
2,2,0,3,1,1,1
3,3,0,1,1,0,1
4,3,1,1,1,0,0


### Data is ready for modeling